In [1]:
from astroquery.sdss import SDSS
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
import math
import numpy as np
from specutils import Spectrum
from specutils.fitting import fit_generic_continuum
from specutils.manipulation import LinearInterpolatedResampler
from labellines import labelLines
import matplotlib.axes as axes
from scipy.optimize import curve_fit
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import scipy
from PIL import Image
from fpdf import FPDF
import random
import pandas as pd
import astropy
import astropy.table
import time
import os

In [2]:
eboss_fits = fits.open('eboss_dr17_summary_edapv10_cigale.fit', memmap=True)

eboss_data = eboss_fits[1].data
eboss_fits.info()
print(eboss_fits[1].columns)

Filename: eboss_dr17_summary_edapv10_cigale.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    235   1898753R x 109C   ['J', 'J', 'J', '16A', 'D', 'E', 'D', 'D', 'E', '118A', '4A', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '9A', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '8E', '8E', '8E', '8E', '5E', '5E', '5E', '5E', 'E', 'E', 'A', 'A', 'A', 'I', 'I', 'I']   
ColDefs(
    name = 'PLATE'; format = 'J'
    name = 'MJD'; format = 'J'
    name = 'FIBER'; format = 'J'
    name = 'PMF_STRING'; format = '16A'
    name = 'Z'; format = 'D'
    name = 'Z_ERR'; format = 'E'
    nam

In [5]:
# Remove possible +/- inf values
gals_flux = eboss_data[(eboss_data['NII_6584_FLUX_ERR']>0)]

err_range = 3

# Make sure relavent emission lines have well measured fluxes
quality_gals = gals_flux[((gals_flux['H_ALPHA_FLUX']/gals_flux['H_ALPHA_FLUX_ERR'])>err_range) &
                (((gals_flux['NII_6584_FLUX']/gals_flux['NII_6584_FLUX_ERR'])>err_range) | (gals_flux["H_GAMMA_EW"]>10)) & 
                (gals_flux['Z']<0.15)] #signal to noise ratio filter

cut_gals = quality_gals[(np.log10(quality_gals['NII_6584_FLUX']/quality_gals['H_ALPHA_FLUX'])<(-1.5))]

print(len(cut_gals))

gals_sf = cut_gals[cut_gals['SFG']==1]
gals_agn = cut_gals[cut_gals['AGN']==1]

1286


C:\Users\nihad\AppData\Local\Temp\ipykernel_28520\3052148939.py:11: RuntimeWarning: divide by zero encountered in log10
  cut_gals = quality_gals[(np.log10(quality_gals['NII_6584_FLUX']/quality_gals['H_ALPHA_FLUX'])<(-1.5))]


In [29]:
for i in range(len(cut_gals)):
    gal = cut_gals[i]
    plate = str(gal["PLATE"])
    mjd = str(gal["MJD"])
    fiber = str(gal["FIBER"])
    if len(fiber) < 4:
        zero_pad = ""
        for i in range(4-len(fiber)):
            zero_pad += "0"
        fiber = zero_pad + fiber
    print("https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/"+plate+"/spec-"+plate + "-" + mjd+ "-"+ fiber+".fits")

https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3589/spec-3589-55186-0368.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3589/spec-3589-55186-0374.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3590/spec-3590-55201-0626.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3615/spec-3615-55856-0090.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3615/spec-3615-56544-0100.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3647/spec-3647-55241-0046.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3647/spec-3647-56568-0042.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3650/spec-3650-55244-0340.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3657/spec-3657-55244-0004.fits
https://data.sdss.org/sas/dr17/eboss/spectro/redux/v5_13_2/spectra/full/3

In [7]:
c = 3*10**5 #km/s
inf = math.inf

def o_iii_1(x, amp, sig, v):
    x_0 = 5008.240* (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2))/0.331)
    
def o_iii_2(x, amp, sig, v):
    x_0 = 4960.295 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def broad_1(x, amp, sig, v):
    x_0 = 5008.240* (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2))/0.331)

def broad_2(x, amp, sig, v):
    x_0 = 4960.295 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def he_i(x, amp, sig, v):
    x_0 = 5017.077 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def fe(x, amp, sig, v):
    x_0 = 4987.20 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def h_beta(x, amp, sig, v):
    x_0 = 4862.68 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def Gauss_oiii(x,B,amp1, amp2, amp3, amp4,amp5, sig1, sig2, v_n, v_b):
    return B+o_iii_1(x, amp1, sig1, v_n) + o_iii_2(x, amp1, sig1, v_n) + broad_1(x, amp5, sig2, v_b) + broad_2(x, amp5, sig2, v_b) + he_i(x, amp2, sig1, v_n) + fe(x, amp3, sig1, v_n)+h_beta(x, amp4, sig1, v_n)

In [9]:
c = 3*10**5 #km/s
inf = math.inf

def o_iii_1(x, amp, sig, v):
    x_0 = 5008.240* (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2))/0.331)
    
def o_iii_2(x, amp, sig, v):
    x_0 = 4960.295 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def he_i(x, amp, sig, v):
    x_0 = 5017.077 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def fe(x, amp, sig, v):
    x_0 = 4987.20 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def h_beta(x, amp, sig, v):
    x_0 = 4862.68 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x - x_0)**2 / (2 * wid**2)))

def Gauss_oiii_no_broad(x,B,amp1, amp2, amp3,amp4, sig, v_n):
    return B+o_iii_1(x, amp1, sig, v_n) + o_iii_2(x, amp1, sig, v_n)+ he_i(x, amp2, sig, v_n) + fe(x, amp3, sig, v_n)+h_beta(x, amp4, sig, v_n)

In [11]:
c = 3*10**5 #km/s
inf = math.inf

def h_alpha(x, amp, sig, v):
    x_0 = 6564.61 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * wid**2)))

def n_ii_1(x, amp, sig, v):
    x_0 = 6549.86 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * wid**2)))

def n_ii_2(x, amp, sig, v):
    x_0 = 6585.27 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * wid**2))/0.326)

def broad(x, amp, sig, v):
    x_0 = 6564.61 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * (wid)**2)))

def Gauss_h_alpha(x, b,amp1, amp2, amp3, sig1, sig2, v_n, v_b):
    return  b + h_alpha(x, amp1, sig1, v_n) + n_ii_1(x, amp2, sig1, v_n) + n_ii_2(x, amp2, sig1, v_n) + broad(x, amp3, sig2, v_b)

In [13]:
c = 3*10**5 #km/s
inf = math.inf

def h_alpha(x, amp, sig, v):
    x_0 = 6564.61 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * wid**2)))

def n_ii_1(x, amp, sig, v):
    x_0 = 6549.86 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * wid**2)))

def n_ii_2(x, amp, sig, v):
    x_0 = 6585.27 * (1+(v/c))
    wid= sig*x_0/c
    return (amp * np.exp(-(x- x_0)**2 / (2 * wid**2))/0.326)

def Gauss_h_alpha_no_broad(x, b,amp1, amp2, sig, v_n):
    return  b + h_alpha(x, amp1, sig, v_n) + n_ii_1(x, amp2, sig, v_n) + n_ii_2(x, amp2, sig, v_n)

In [23]:
results_broad= pd.DataFrame()
results_nobroad= pd.DataFrame()
# Record the start time
start_time = time.time()  # Or time.perf_counter() for higher resolution

for i in range(len(cut_gals)):
    print(i)
    gal = cut_gals[i]
    plate = str(gal["PLATE"])
    mjd = str(gal["MJD"])
    fiber = str(gal["FIBER"])
    if len(fiber) < 4:
        zero_pad = ""
        for j in range(4-len(fiber)):
            zero_pad += "0"
        fiber = zero_pad + fiber
    print(plate + "-" + mjd+ "-"+ fiber)
    fits_directory = "C:/Users/nihad/pipelines/"

    # Define the name of your FITS file
    fits_filename = "spec-"+plate+"-"+mjd+"-"+fiber+".fits"
    
    # Construct the full path to the FITS file
    fits_path = os.path.join(fits_directory, fits_filename)
    spec_hdu = fits.open(fits_path, memmap=True)
    z = spec_hdu[2].data["z"]
    spec_data = spec_hdu[1].data
    header = spec_hdu[0].header

    loglam = spec_data['loglam'].copy()
    flux = spec_data['flux'].copy()
    wavelength = 10**loglam/(1+z)
    ivar = spec_data['ivar'].copy()

    max_y = []
    flux_sum =[]
    wave_sum = []
    ivar_sum = []
    filter = []
    nexp = header['NEXP']
    for sub_exp in range(nexp//2):
        sub_data = spec_hdu[4+sub_exp].data
        sub_flux = sub_data['flux']
        sub_wavelength = 10**sub_data['loglam'] / (1+z)
        sub_ivar = sub_data['ivar'] 
        flux_sum.extend(sub_flux[sub_ivar>0])
        wave_sum.extend(sub_wavelength[sub_ivar>0])
        ivar_sum.extend(sub_ivar[sub_ivar>0])
        filter.extend([sub_exp for _ in range(len(sub_flux[sub_ivar>0]))])

    combined = sorted(zip(wave_sum, flux_sum,ivar_sum,filter))
    # Unzip the sorted pairs back into separate lists
    sorted_wave, sorted_flux, sorted_ivar, sorted_filter = zip(*combined) 

    min = np.where(np.array(sorted_wave) > 4600)[0][0]
    max = np.where(np.array(sorted_wave) > 5400)[0][0]

    inf = math.inf
    ranged_wave = sorted_wave[min:max]
    ranged_flux = sorted_flux[min:max]
    ranged_ivar = sorted_ivar[min:max]

    parameters, covariance = curve_fit(Gauss_oiii, ranged_wave, ranged_flux,sigma = 1/np.sqrt(sorted_ivar[min:max]),  method = "trf", p0=[2, 200, 2, 2,2,2, 50, 1000, 50, 50], bounds=([0,0,0,0,0,0,20,200,-200,-2000],[10,1000,50,5,1000,5,200, 3000,200,2000]))
    B, amp1, amp2, amp3, amp4, amp5, sig1, sig2, v_n, v_b= parameters
    err = np.sqrt(np.diag(covariance))

    fit_y = Gauss_oiii(ranged_wave, B, amp1, amp2, amp3, amp4, amp5, sig1, sig2, v_n, v_b)
    
    err = (ranged_flux-fit_y)
    chi_square = np.square(err)*ranged_ivar
    BIC = sum(chi_square) - 10*np.log(len(ranged_flux))
    
    print(i)
    results_broad.loc[i, 'plate'] = gal["PLATE"]
    results_broad.loc[i, 'mjd'] = gal["MJD"]
    results_broad.loc[i, 'fiber'] = gal["FIBER"]
    results_broad.loc[i, 'oiii_continuuum'] = B
    results_broad.loc[i, 'oiii_continuuum_err'] = err[0]
    results_broad.loc[i, 'oiii'] = amp1
    results_broad.loc[i, 'oiii_err'] = err[1]
    results_broad.loc[i, 'he_i'] = amp2
    results_broad.loc[i, 'he_i_err'] = err[2]
    results_broad.loc[i, 'fe_i/ii'] = amp3
    results_broad.loc[i, 'fe_i/ii_err'] = err[3]
    results_broad.loc[i, 'h_beta'] = amp4
    results_broad.loc[i, 'h_beta_err'] = err[4]
    results_broad.loc[i, 'broad_oiii'] = amp5
    results_broad.loc[i, 'broad_oiii_err'] = err[5]
    results_broad.loc[i, 'sigma1_oiii'] = sig1
    results_broad.loc[i, 'sigma1_oiii_err'] = err[6]
    results_broad.loc[i, 'sigma2_oiii'] = sig2
    results_broad.loc[i, 'sigma2_oiii_err'] = err[7]
    results_broad.loc[i, 'v_n_oiii'] = v_n
    results_broad.loc[i, 'v_n_oiii_err'] = err[8]
    results_broad.loc[i, 'v_b_oiii'] = v_b
    results_broad.loc[i, 'v_b_oiii_err'] = err[9]
    results_broad.loc[i, 'BIC_oiii'] = BIC

    parameters, covariance = curve_fit(Gauss_oiii_no_broad, ranged_wave, ranged_flux,sigma = 1/np.sqrt(sorted_ivar[min:max]),  method = "trf", p0=[2, 200, 2, 2,2, 50, 50], bounds=([0,0,0,0,0,20,-200],[10,1000,50,5,1000,200,200]))
    B, amp1, amp2, amp3, amp4,  sig, v_n= parameters
    err = np.sqrt(np.diag(covariance))

    fit_y = Gauss_oiii_no_broad(ranged_wave, B, amp1, amp2, amp3, amp4,  sig, v_n)
    
    err = (ranged_flux-fit_y)
    chi_square = np.square(err)*ranged_ivar
    BIC = sum(chi_square) - 7*np.log(len(ranged_flux))

    results_nobroad.loc[i, 'plate'] = gal["PLATE"]
    results_nobroad.loc[i, 'mjd'] = gal["MJD"]
    results_nobroad.loc[i, 'fiber'] = gal["FIBER"]
    results_nobroad.loc[i, 'oiii_continuuum'] = B
    results_nobroad.loc[i, 'oiii_continuuum_err'] = err[0]
    results_nobroad.loc[i, 'oiii'] = amp1
    results_nobroad.loc[i, 'oiii_err'] = err[1]
    results_nobroad.loc[i, 'he_i'] = amp2
    results_nobroad.loc[i, 'he_i'] = err[2]
    results_nobroad.loc[i, 'fe_i/ii'] = amp3
    results_nobroad.loc[i, 'fe_i/ii_err'] = err[3]
    results_nobroad.loc[i, 'h_beta'] = amp4
    results_nobroad.loc[i, 'h_beta_err'] = err[4]
    results_nobroad.loc[i, 'sigma_oiii'] = sig
    results_nobroad.loc[i, 'sigma_oiii_err'] = err[5]
    results_nobroad.loc[i, 'v_n_oiii'] = v_n
    results_nobroad.loc[i, 'v_n_oiii_err'] = err[6]
    results_nobroad.loc[i, 'BIC_oiii'] = BIC
    
    max_y = []
    flux_sum =[]
    wave_sum = []
    ivar_sum = []
    filter = []
    nexp = header['NEXP']
    for sub_exp in range(nexp//2):
        sub_data = spec_hdu[4+nexp//2+sub_exp].data
        sub_flux = sub_data['flux']
        sub_wavelength = 10**sub_data['loglam'] / (1+z)
        sub_ivar = sub_data['ivar'] 
        flux_sum.extend(sub_flux[sub_ivar>0])
        wave_sum.extend(sub_wavelength[sub_ivar>0])
        ivar_sum.extend(sub_ivar[sub_ivar>0])
        filter.extend([sub_exp for _ in range(len(sub_flux[sub_ivar>0]))])

    combined = sorted(zip(wave_sum, flux_sum,ivar_sum,filter))
    # Unzip the sorted pairs back into separate lists
    sorted_wave, sorted_flux, sorted_ivar, sorted_filter = zip(*combined) 

    min = np.where(np.array(sorted_wave) > 6000)[0][0]
    max = np.where(np.array(sorted_wave) > 7000)[0][0]

    inf = math.inf
    ranged_wave = sorted_wave[min:max]
    ranged_flux = sorted_flux[min:max]
    ranged_ivar = sorted_ivar[min:max]

    parameters, covariance = curve_fit(Gauss_h_alpha, ranged_wave, ranged_flux,sigma = 1/np.sqrt(sorted_ivar[min:max]),  method = "trf", 
                                   p0=[2, 200, 2, 2, 50, 1000, 50, 50], bounds=([0,0,0,0,20,200,-200,-2000],[10,1000,50,50,200, 4000,200,2000]))
    B, amp1, amp2, amp3, sig1, sig2, v_n, v_b= parameters
    err = np.sqrt(np.diag(covariance))
    fit_y = Gauss_h_alpha(ranged_wave, B, amp1, amp2, amp3, sig1, sig2, v_n, v_b)
    
    err = (ranged_flux-fit_y)
    chi_square = np.square(err)*ranged_ivar
    BIC = sum(chi_square) - 8*np.log(len(ranged_flux))

    
    results_broad.loc[i, 'h_continuuum'] = B
    results_broad.loc[i, 'h_continuuum_err'] = err[0]
    results_broad.loc[i, 'h_alpha'] = amp1
    results_broad.loc[i, 'h_alpha_err'] = err[1]
    results_broad.loc[i, 'n_ii'] = amp2
    results_broad.loc[i, 'n_ii_err'] = err[2]
    results_broad.loc[i, 'broad_h'] = amp3
    results_broad.loc[i, 'broad_h_err'] = err[3]
    results_broad.loc[i, 'sigma1_h'] = sig1
    results_broad.loc[i, 'sigma1_h_err'] = err[4]
    results_broad.loc[i, 'sigma2_h'] = sig2
    results_broad.loc[i, 'sigma2_h_err'] = err[5]
    results_broad.loc[i, 'v_n_h'] = v_n
    results_broad.loc[i, 'v_n_h_err'] = err[6]
    results_broad.loc[i, 'v_b_h'] = v_b
    results_broad.loc[i, 'v_b_h_err'] = err[7]
    results_broad.loc[i, 'BIC_h'] = BIC

    parameters, covariance = curve_fit(Gauss_h_alpha_no_broad, ranged_wave, ranged_flux,sigma = 1/np.sqrt(sorted_ivar[min:max]),  method = "trf", 
                                   p0=[2, 200, 2, 50, 50], bounds=([0,0,0,20,-200],[10,1000,20,200, 200]))
    B, amp1, amp2, sig, v_n= parameters
    err = np.sqrt(np.diag(covariance))

    fit_y = Gauss_h_alpha_no_broad(ranged_wave, B, amp1, amp2, sig, v_n)
    
    err = (ranged_flux-fit_y)
    chi_square = np.square(err)*ranged_ivar
    BIC = sum(chi_square) - 5*np.log(len(ranged_flux))


    results_nobroad.loc[i, 'h_continuuum'] = B
    results_nobroad.loc[i, 'h_continuuum_err'] = err[0]
    results_nobroad.loc[i, 'h_alpha'] = amp1
    results_nobroad.loc[i, 'h_alpha_err'] = err[1]
    results_nobroad.loc[i, 'n_ii'] = amp2
    results_nobroad.loc[i, 'n_ii_err'] = err[2]
    results_nobroad.loc[i, 'sigma_h'] = sig
    results_nobroad.loc[i, 'sigma_h_err'] = err[3]
    results_nobroad.loc[i, 'v_n_h'] = v_n
    results_nobroad.loc[i, 'v_n_h_err'] = err[4]
    results_nobroad.loc[i, 'BIC_h'] = BIC

# Record the end time
end_time = time.time()  # Or time.perf_counter()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

0
3589-55186-0368
0
1
3589-55186-0374
1
2
3590-55201-0626
2
3
3615-55856-0090
3
4
3615-56544-0100
4
5
3647-55241-0046
5
6
3647-56568-0042
6
7
3650-55244-0340
7
8
3657-55244-0004
8
9
3658-55205-0474
9
10
3660-55209-0711
10
11
3663-55176-0130
11
12
3663-55176-0387
12
13
3663-55176-0766
13
14
3669-55481-0572
14
15
3670-55480-0196
15
16
3679-55209-0234
16
17
3682-55244-0430
17
18
3691-55274-0169
18
19
3696-55277-0898
19
20
3699-55517-0486
20
21
3736-55214-0168
21
22
3736-55214-0352
22
23
3744-55209-0368
23
24
3756-55505-0496
24
25
3758-55506-0040
25
26
3764-55514-0998
26
27
3768-55564-0306
27
28
3769-55240-0256
28
29
3770-55234-0150
29
30
3771-55544-0136
30
31
3771-55544-0580
31
32
3771-55544-0642
32
33
3771-55544-0724
33
34
3772-55277-0036
34
35
3772-55277-0770
35
36
3772-55277-0848
36
37
3781-55243-0697
37
38
3784-55269-0154
38
39
3785-55241-0946
39
40
3785-55273-0942
40
41
3801-55509-0412


KeyboardInterrupt: 

In [21]:
results_broad = results_broad.astype({'plate': int, 'mjd': int, 'fiber': int})
results_broad.info()
table = astropy.table.Table.from_pandas(results_broad)
table.write("parameters_w_BIC.fits", overwrite = True)

results_nobroad = results_nobroad.astype({'plate': int, 'mjd': int, 'fiber': int})
results_nobroad.info()
table = astropy.table.Table.from_pandas(results_nobroad)
table.write("parameters_nobroad_w_BIC.fits", overwrite = True)


<class 'pandas.core.frame.DataFrame'>
Index: 1286 entries, 0 to 1285
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   plate                1286 non-null   int32  
 1   mjd                  1286 non-null   int32  
 2   fiber                1286 non-null   int32  
 3   oiii_continuuum      1286 non-null   float64
 4   oiii_continuuum_err  1286 non-null   float32
 5   oiii                 1286 non-null   float64
 6   oiii_err             1286 non-null   float32
 7   he_i                 1286 non-null   float64
 8   he_i_err             1286 non-null   float32
 9   fe_i/ii              1286 non-null   float64
 10  fe_i/ii_err          1286 non-null   float32
 11  h_beta               1286 non-null   float64
 12  h_beta_err           1286 non-null   float32
 13  broad_oiii           1286 non-null   float64
 14  broad_oiii_err       1286 non-null   float32
 15  sigma1_oiii          1286 non-null   float6